In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [2]:
def load_glove_model(glove_file_path):
    glove_model = KeyedVectors.load_word2vec_format(glove_file_path, binary=False, no_header=True)
    return glove_model

def get_corpus():
    with open("../data/processed/glove_corpus", "r") as f:
        sentences = f.readlines()
    return sentences

In [3]:
sentences = get_corpus()
tokens = [sent.split() for sent in sentences]

In [4]:
base_model = Word2Vec(vector_size=300, min_count = 20, epochs=20)
base_model.build_vocab(tokens)
total_examples = base_model.corpus_count

In [5]:
base_model.wv.most_similar('girl', topn=10)

[('entertainment', 0.22759167850017548),
 ('weeks,', 0.19057254493236542),
 ('unless', 0.17623214423656464),
 ('treatment', 0.1741826832294464),
 ('wooden', 0.17394684255123138),
 ('learn', 0.17307960987091064),
 ('point.', 0.17082807421684265),
 ('welcome.', 0.17074212431907654),
 ('beijing.', 0.17042849957942963),
 ('administrative', 0.1670820116996765)]

In [6]:
corpus_path = '../embeds/GloVe/glove.840B.300d.txt'
corpus_model = load_glove_model(corpus_path)
base_model.build_vocab([list(corpus_model.key_to_index.keys())], update=True)

In [7]:
corpus_model.most_similar('girl', topn=10)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


[('girls', 0.8245455622673035),
 ('boy', 0.8148322105407715),
 ('woman', 0.7700793147087097),
 ('lady', 0.7557979226112366),
 ('teen', 0.7407615780830383),
 ('sexy', 0.7246244549751282),
 ('blonde', 0.6959359645843506),
 ('teenage', 0.6914814114570618),
 ('chick', 0.6885996460914612),
 ('babe', 0.688468873500824)]

In [8]:
fine_model = Word2Vec(vector_size=300, min_count = 20, epochs=20)
fine_model.build_vocab(tokens)

In [9]:
fine_model.train(tokens, total_examples=total_examples, epochs=fine_model.epochs)
fine_model_wv = fine_model.wv

In [10]:
fine_model_wv.most_similar('look', topn=10)

[('looked', 0.4677370488643646),
 ('looking', 0.44000688195228577),
 ('looks', 0.4197654128074646),
 ('fit', 0.40606167912483215),
 ('move', 0.39111045002937317),
 ('moment.', 0.34159228205680847),
 ('all!', 0.3370228111743927),
 ('try', 0.33682525157928467),
 ('pick', 0.3343583047389984),
 ('last.', 0.3326244354248047)]

In [ ]:
base_model_wv.most_similar('does', topn=10)

[("doesn't", 0.5743285417556763),
 ('did', 0.40398046374320984),
 ('do', 0.38602036237716675),
 ("doesn't.", 0.3456938564777374),
 ('does,', 0.34460973739624023),
 ('is', 0.3222086429595947),
 ('needs', 0.3137502670288086),
 ('prefers', 0.3080991506576538),
 ('makes', 0.3018656075000763),
 ('likes', 0.2977132797241211)]

In [17]:
fine_model.wv.most_similar("pleasure")

[('pleasure.', 0.5820543766021729),
 ('nice', 0.4965497553348541),
 ('pleasure,', 0.47940748929977417),
 ('pleasant', 0.41643211245536804),
 ('tom.', 0.39964956045150757),
 ('great', 0.3727753758430481),
 ('grandma', 0.37242355942726135),
 ('lovely', 0.3650278151035309),
 ('thoughtful', 0.36045509576797485),
 ('talent', 0.35631096363067627)]

In [16]:
fine_model.wv.most_similar("birthday")

[('birthday.', 0.6442477703094482),
 ('wedding', 0.6132053732872009),
 ('birthday,', 0.5925280451774597),
 ("wife's", 0.5911794900894165),
 ('surprise', 0.526892364025116),
 ("mother's", 0.5216265320777893),
 ('party?', 0.5086395144462585),
 ("sister's", 0.5069356560707092),
 ("mom's", 0.5007989406585693),
 ('birthday?', 0.4949586093425751)]

In [15]:
fine_model.wv.most_similar("wonderful")

[('great', 0.6634839177131653),
 ('lovely', 0.5981692671775818),
 ('fantastic', 0.5609707832336426),
 ('beautiful', 0.5124997496604919),
 ('pleasant', 0.4743961989879608),
 ('perfect', 0.45208853483200073),
 ('good', 0.4515893757343292),
 ('fun', 0.4398481547832489),
 ('nice', 0.4351896345615387),
 ('clever', 0.4205162525177002)]